# Bitcoin Price - Sentiment analysis

In [18]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from nltk import word_tokenize
import requests
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from collections import OrderedDict
import pprint
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from operator import itemgetter
import gensim.summarization
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from wordcloud import WordCloud, STOPWORDS
from sklearn import linear_model
import statsmodels.api as sm
from textblob import TextBlob

In [19]:
# Data Source - https://www.kaggle.com/skularat/bitcoin-tweets
df = pd.read_csv("BitcoinTweets 2.csv")

In [20]:
df.head()

table_key            tweet_id  \
0          1  892535404292968448   
1          2  892535387280867328   
2          3  892535372139249664   
3          4  892535371958898688   
4          5  892535353063731200   

                                                text        date  favorites  \
0  #bitcoin #bitcoinfork #hardfork #bitcoincash p...  1501631940          2   
1  #bitcoin : Whats going on with the digital cur...  1501631940          0   
2  From the #Bitcoin White Paper: "We define an e...  1501631940          1   
3  #Bitcoin : What's going on with the #digitalcu...  1501631940          5   
4  Found an app called @SlideCoin that pays #bitc...  1501631940          0   

   retweets  
0         0  
1         0  
2         0  
3         0  
4         0

# Sentiment Vader Anlaysis

In [21]:
def clean_tweet(sentence):
    words = word_tokenize(sentence)
    lowercase_words = [word.lower() for word in words
                       if word not in stopwords.words() and word.isalpha()]
    temp_text = ' '.join(lowercase_words)
    return temp_text

In [22]:
def get_tweet_sentiment(tweet): 
    analysis = TextBlob(clean_tweet(tweet)) 
    if (analysis.sentiment.polarity > 0): 
        return 1
    elif (analysis.sentiment.polarity == 0): 
        return 0
    else:
        return -1

In [23]:
get_tweet_sentiment(df['text'][10])

0

In [24]:
def vader_comparison(sentence):
    sentence=clean_tweet(sentence)
    analyzer = SentimentIntensityAnalyzer()
    pos=compound=neu=neg=0
    vs = analyzer.polarity_scores(sentence)
    pos = vs['pos']
    neu = vs['neu']
    neg = vs['neg']
    compound = vs['compound']
    if (compound > 0):
        return 1
    elif (compound == 0):
        return 0
    else:
        return -1

In [25]:
vader_comparison(df['text'][1000])

0

In [26]:
get_tweet_sentiment(df['text'][1000])

1

In [27]:
df['score'] = 0.75*df['retweets'] + 0.25*df['favorites']
df = df.sort_values(['score'], ascending=False)
df = df[df['score'] > 2.5]
df1 = df.sample(n=100000)

In [28]:
df1['score'].describe()

count    100000.000000
mean         27.684575
std         204.729134
min           2.750000
25%           3.750000
50%           6.000000
75%          14.750000
max       49242.750000
Name: score, dtype: float64

In [ ]:
%%timeit
df1['sentiment_textblob'] = df1['text'].apply(lambda x: get_tweet_sentiment(x))

In [ ]:
%%timeit
df1['sentiment_vader'] = df1['text'].apply(lambda x: vader_comparison(x))

In [30]:
#The above two pieces of code a long time to compute so saved a new cleaned file "cleaned.csv" to use the data without having to compute everytime
df1 = pd.read_csv('cleaned.csv')

In [31]:
df1

Unnamed: 0  table_key            tweet_id  \
0        5226652    5358842  913895212627451906   
1        1345503    1378655  922387557463347200   
2        2365781    2422956  852934818329395201   
3         127702     130232  913682756609105920   
4        5414150    5554459  968942863093587968   
5        3877777    3971660  868196571187486720   
6        5965384    6134760  833676388263657472   
7        3747885    3839083  956749000773812226   
8        2386370    2443867  951581515846246400   
9         318170     323677  915340689491349506   
10        331129     336774  925542412310720512   
11         33858      34647  895247327526518785   
12       4567148    4680984  906802303210057728   
13       1301508    1333566  932893024509681664   
14       4446909    4557062  874395170300600320   
15       1108131    1136399  921088955113725952   
16        350744     356909  937422565856948224   
17       2087782    2139462  839223130266746881   
18       1778481    1823748  947388862082568192   
19       6017355    6189814  834896421941751810   
20       2765888    2829771  952992506496667648   
21       3924594    4019684  900090888457777152   
22       3402136    3484359  955432944130240513   
23       5033009    5159027  880401746127966208   
24       5180066    5310666  912874251526590465   
25       3558603    3646212  896236204705718272   
26       3548116    3635249  864895153697419264   
27       3564190    3651809  864765433995460608   
28       3716316    3807040  897874314854322178   
29       1374654    1408485  934473978302943232   
...          ...        ...                 ...   
9970     4988120    5113037  880068439204933632   
9971     3221385    3298280  892274727074639872   
9972     2749778    2813415  855964038752124928   
9973     5200309    5331767  964779979698462720   
9974     5980514    6150912  834101124596592640   
9975     1082400    1110003  942058310903181312   
9976     5551929    5698992  823352704395137024   
9977     1598250    1637347  925083450599727109   
9978     4383270    4491628  960589689198710792   
9979     5508094    5653007  822475319458299906   
9980     1652679    1692765  944995909372227584   
9981      357255     363857  926131899126484994   
9982     4680044    4797502  961531216443920384   
9983     2978177    3047731  953683345845571585   
9984     2302975    2358883  841035322590920706   
9985     5459655    5602531  821458588660969473   
9986      253111     257732  925097243425869826   
9987      500772     511367  938868392567738369   
9988     1732141    1775755  946760050294644736   
9989     4172420    4274535  959438775792422918   
9990     2109385    2161359  949601484689338373   
9991     1563554    1601895  944116363294937088   
9992     5391356    5530385  819383197867720704   
9993     2510386    2570547  951930374833229824   
9994      488366     498643  938900336319893504   
9995     1654793    1694902  944983627074486272   
9996     3451033    3534742  895331807847079936   
9997     1984542    2033998  948999582440071168   
9998      916895     941315  929150300228898816   
9999     2251532    2306106  840020139684716544   

                                                   text        date  \
0     # Bitcoin Current bitcoin price is 0.00023961 ...  1506724560   
1     J'ai investi dans le # Bitcoin - Mon avis sur ...  1508749260   
2     Find one of 5 Byteball Easter Eggs each worth ...  1492190460   
3     Name of my @ openbazaar store Ready fir the Cr...  1506673860   
4     # celebrity # Gossip # GOESS # askvpmc # YouTu...  1519848900   
5     Even the worlds largest # bitcoin exchange cou...  1495829160   
6     Top 6 # Bitcoin Trading Bots http://ow.ly/SAxz...  1487598900   
7     # Bitcoin Takes Davos Stage in # Currency Pane...  1516941660   
8     IBM and Stellar launch blockchain banking acro...  1515709680   
9     More on GS: Goldman Sachs exploring # bitcoin ...  1507069200   
10    Buy # bitcoins for # cash here at installed @ ...  15095

In [32]:
def combine():
    tweet1 = ''
    for i in range(0, len(df1)):
        tweet1 = tweet1 + '\n' + df1['text'][i]
    tweets = tweet1.replace('\n\n', ' ')
    tweets = tweets.replace('\n', ' ')
    tweets = ''.join(tweets)
    words = word_tokenize(tweets)
    lowercase = [word.lower() for word in words
                 if word not in stopwords.words() and word.isalpha()]
    tweets = ' '.join(lowercase)
    return tweets

In [33]:
def wordcloud():
    tweets = combine()
    print(tweets)
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',max_words=20, collocations=False).generate(tweets)
    plt.axis('off')
    plt.imshow(wordcloud)
    plt.show()

In [ ]:
# Drawing Word Cloud
wordcloud()

In [36]:
def draw_wordcloud(lda,topicnum,min_size=0,STOPWORDS=[]):
    word_list=[]
    prob_total = 0
    for word,prob in lda.show_topic(topicnum,topn=50):
        prob_total +=prob
    for word,prob in lda.show_topic(topicnum,topn=50):
        if word in STOPWORDS or  len(word) < min_size:
            continue
        freq = int(prob/prob_total*1000)
        alist=[word]
        word_list.extend(alist*freq)

    text = ' '.join(word_list)
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',max_words=20, collocations=False).generate(text)

    plt.axis('off')
    plt.imshow(wordcloud)
    plt.show()
    return None

In [37]:
def topic():
    text = combine()
    new_text = [nltk.word_tokenize(text)]
    dictionary = corpora.Dictionary(new_text) 
    corpus = [dictionary.doc2bow(text) for text in new_text]
    lda = LdaModel(corpus, id2word=dictionary, num_topics=5, passes=10)
    lda.get_document_topics(corpus,minimum_probability=0.05)
    sorted(lda.get_document_topics(corpus,minimum_probability=0),key=itemgetter(1),reverse=True)
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(lda.print_topics(num_words=8))
    draw_wordcloud(lda,4)
    return lda, corpus, dictionary

In [38]:
#Topic analysis
lda, corpus, dictionary = topic()

[   (   0,
        '0.003*"bitcoin" + 0.001*"http" + 0.001*"blockchain" + 0.001*"https" + '
        '0.001*"cryptocurrency" + 0.001*"btc" + 0.000*"crypto" + '
        '0.000*"fintech"'),
    (   1,
        '0.004*"bitcoin" + 0.001*"blockchain" + 0.001*"https" + 0.001*"http" + '
        '0.001*"crypto" + 0.001*"btc" + 0.001*"cryptocurrency" + '
        '0.000*"fintech"'),
    (   2,
        '0.085*"bitcoin" + 0.025*"http" + 0.023*"https" + 0.019*"blockchain" + '
        '0.013*"btc" + 0.012*"cryptocurrency" + 0.011*"crypto" + '
        '0.008*"ethereum"'),
    (   3,
        '0.004*"bitcoin" + 0.002*"http" + 0.002*"https" + 0.001*"btc" + '
        '0.001*"blockchain" + 0.001*"cryptocurrency" + 0.001*"crypto" + '
        '0.001*"fintech"'),
    (   4,
        '0.002*"bitcoin" + 0.001*"https" + 0.001*"blockchain" + 0.000*"http" + '
        '0.000*"cryptocurrency" + 0.000*"btc" + 0.000*"crypto" + '
        '0.000*"ethereum"')]


In [42]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [5]:
#source - https://www.cryptodatadownload.com
bitcoin = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2018-11-11.csv')

In [6]:
bitcoin

Timestamp     Open     High      Low    Close  Volume_(BTC)  \
0        1325317920     4.39     4.39     4.39     4.39      0.455581   
1        1325317980      NaN      NaN      NaN      NaN           NaN   
2        1325318040      NaN      NaN      NaN      NaN           NaN   
3        1325318100      NaN      NaN      NaN      NaN           NaN   
4        1325318160      NaN      NaN      NaN      NaN           NaN   
5        1325318220      NaN      NaN      NaN      NaN           NaN   
6        1325318280      NaN      NaN      NaN      NaN           NaN   
7        1325318340      NaN      NaN      NaN      NaN           NaN   
8        1325318400      NaN      NaN      NaN      NaN           NaN   
9        1325318460      NaN      NaN      NaN      NaN           NaN   
10       1325318520      NaN      NaN      NaN      NaN           NaN   
11       1325318580      NaN      NaN      NaN      NaN           NaN   
12       1325318640      NaN      NaN      NaN      NaN           NaN   
13       1325318700      NaN      NaN      NaN      NaN           NaN   
14       1325318760      NaN      NaN      NaN      NaN           NaN   
15       1325318820      NaN      NaN      NaN      NaN           NaN   
16       1325318880      NaN      NaN      NaN      NaN           NaN   
17       1325318940      NaN      NaN      NaN      NaN           NaN   
18       1325319000      NaN      NaN      NaN      NaN           NaN   
19       1325319060      NaN      NaN      NaN      NaN           NaN   
20       1325319120      NaN      NaN      NaN      NaN           NaN   
21       1325319180      NaN      NaN      NaN      NaN           NaN   
22       1325319240      NaN      NaN      NaN      NaN           NaN   
23       1325319300      NaN      NaN      NaN      NaN           NaN   
24       1325319360      NaN      NaN      NaN      NaN           NaN   
25       1325319420      NaN      NaN      NaN      NaN           NaN   
26       1325319480      NaN      NaN      NaN      NaN           NaN   
27       1325319540      NaN      NaN      NaN      NaN           NaN   
28       1325319600      NaN      NaN      NaN      NaN           NaN   
29       1325319660      NaN      NaN      NaN      NaN           NaN   
...             ...      ...      ...      ...      ...           ...   
3603106  1541892600  6353.02  6354.28  6353.02  6354.28      2.312530   
3603107  1541892660  6354.28  6361.13  6354.28  6355.37      7.027648   
3603108  1541892720  6354.99  6354.99  6354.98  6354.98      0.540241   
3603109  1541892780  6354.96  6354.98  6354.96  6354.98      0.129379   
3603110  1541892840  6351.43  6354.98  6347.12  6354.10      0.824676   
3603111  1541892900  6354.12  6354.12  6350.40  6350.40      0.561571   
3603112  1541892960  6351.03  6351.03  6349.59  6349.59      0.325010   
3603113  1541893020  6348.70  6354.48  6348.70  6354.47      1.464615   
3603114  1541893080  6353.05  6353.05  6353.05  6353.05      0.122300   
3603115  1541893140  6353.05  6353.05  6353.05  6353.05      0.003553   
3603116  1541893200      NaN      NaN      NaN      NaN           NaN   
3603117  1541893260      NaN      NaN      NaN      NaN           NaN   
3603118  1541893320  6354.48  6354.81  6347.52  6347.52      0.879954   
3603119  1541893380  6354.87  6354.87  6354.87  6354.87      0.021846   
3603120  1541893440  6353.05  6354.32  6353.05  6354.32      0.007991   
3603121  1541893500  6355.23  6355.23  6343.90  6343.90      1.481242   
3603122  1541893560      NaN      NaN      NaN      NaN           NaN   
3603123  1541893620  6343.92  6343.92  6343.92  6343.92      0.113460   
3603124  1541893680  6348.07  6348.07  6348.07  6348.07      0.025751   
3603125  1541893740      NaN      NaN      NaN      NaN           NaN   
3603126  1541893800  6347.60  6347.60  6347.60  6347.60      0.643544   
3603127  1541893860      NaN      NaN      NaN      NaN           NaN   
3603128  1541893920  6348.54  6348.54  6348.54  6348.54      0.07941

In [7]:
bitcoin = bitcoin.dropna()

In [9]:
bitcoin['Timestamp'] = bitcoin['Timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df1['Timestamp'] = df1['date'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [11]:
df1.set_index('Timestamp', inplace=True)
bitcoin.set_index('Timestamp', inplace=True)

In [14]:
df1 = df1.sort_values('Timestamp')

In [16]:
df1.index = pd.to_datetime(df1.index)
bitcoin.index = pd.to_datetime(bitcoin.index)

In [17]:
df1.index = df1.index.values.astype('<M8[m]')
bitcoin.index = bitcoin.index.values.astype('<M8[m]')

# Calculating 2-day moving average of percent change

In [18]:
df1['30_min_prev_avg'] = np.nan
df1['30_min_next_avg'] = np.nan

In [19]:
for i in df1.index:
    start_index = i - timedelta(minutes=30)
    avg_prev  = np.mean(bitcoin[(bitcoin.index >= start_index) & (bitcoin.index <=i)]['Close'])
    df1['30_min_prev_avg'].loc[i] = avg_prev
    last_index = i+ timedelta(minutes=30)
    avg_next = np.mean(bitcoin[(bitcoin.index >= i) & (bitcoin.index <= last_index)]['Close'])
    df1['30_min_next_avg'].loc[i] = avg_next

/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
df1['avg_diff'] = df1['30_min_prev_avg'] - df1['30_min_next_avg']

In [21]:
df1['pos_sent'] = np.repeat(0, len(df1))
df1['neg_sent'] = np.repeat(0, len(df1))

In [22]:
df1.head()

Unnamed: 0  table_key            tweet_id  \
2017-01-01 02:36:00     5221343    5353345  815386239108952066   
2017-01-01 05:04:00     5219732    5351717  815423304823701504   
2017-01-01 06:32:00     5218931    5350916  815445449494724608   
2017-01-01 10:22:00     5216773    5348758  815503487798743040   
2017-01-01 15:02:00     5212864    5344614  815574018761822209   

                                                                  text  \
2017-01-01 02:36:00  There are 3 eras of currency: commodity based ...   
2017-01-01 05:04:00  How to begin building your Crowdify Club onlin...   
2017-01-01 06:32:00  # Bitcoin: 2016s best-performing commodity via...   
2017-01-01 10:22:00  # Bitcoin has been trading at a discount to it...   
2017-01-01 15:02:00  # Bitcoin News What Will the Bitcoin Price Be ...   

                           date  favorites  retweets  score  sentiment_vader  \
2017-01-01 02:36:00  1483238160          3         6   5.25                0   
2017-01-01 05:04:00  1483247040         10        19  16.75               -1   
2017-01-01 06:32:00  1483252320          2         5   4.25                0   
2017-01-01 10:22:00  1483266120         12         9   9.75                0   
2017-01-01 15:02:00  1483282920         17        31  27.50                0   

                     30_min_prev_avg  30_min_next_avg  avg_diff  pos_sent  \
2017-01-01 02:36:00       964.590625       963.763158  0.827467         0   
2017-01-01 05:04:00       963.033750       962.955000  0.078750         0   
2017-01-01 06:32:00       964.000000       964.312000 -0.312000         0   
2017-01-01 10:22:00       967.626190       970.164800 -2.538610         0   
2017-01-01 15:02:00       974.837895       975.974286 -1.136391         0   

                     neg_sent  
2017-01-01 02:36:00         0  
2017-01-01 05:04:00         0  
2017-01-01 06:32:00         0  
2017-01-01 10:22:00         0  
2017-01-01 15:02:00         0

In [23]:
for index, row in df1.iterrows():
    if row['sentiment_vader'] == 1:
        df1.set_value(index,'pos_sent',1)
    elif row['sentiment_vader'] == -1:
        df1.set_value(index,'neg_sent',1)

/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
df1['avg_diff'].median()

-0.8062931034486951

# OLS Regression between sentiment score and price change

In [ ]:
y = df1['avg_diff'].values.copy()
x1 = df1['pos_sent'].values.copy()
x2 = df1['neg_sent'].values.copy()
X = np.array(list(zip(x1, x2)))
model = linear_model.LinearRegression()
model.fit(X, y)

In [42]:
X = df1[['pos_sent','neg_sent','score']]
X = sm.add_constant(X)
y = df1['avg_diff']
model = sm.OLS(y,X,missing='drop')
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               avg_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.056
Date:                Mon, 03 Dec 2018   Prob (F-statistic):              0.366
Time:                        22:02:36   Log-Likelihood:                -59457.
No. Observations:               10000   AIC:                         1.189e+05
Df Residuals:                    9996   BIC:                         1.190e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8496      1.317     -1.404      0.1

# Naive analysis: Check to see if sentiment score corrleates to price fluctuation

In [ ]:
for i in df1.index:
    start_index = i - timedelta(minutes=30)
    avg_prev  = np.mean(df1[(df1.index >= start_index) & (df1.index <=i)]['sentiment_vader'])
    last_index = i+ timedelta(minutes=30)
    avg_next = np.mean(df1[(df1.index >= i) & (df1.index <= last_index)]['sentiment_vader'])
    senti_change = avg_next - avg_prev
    if senti_change >= epsilon:
        df1['Predicted_price_change'] = 1
    elif 0 <= senti_change < epsilon:
        df1['Predicted_price_change'] = 0
    elif senti_change < 0:
        df1['Predicted_price_change'] = -1

In [31]:
df1['Predicted_price_change'] = np.nan

In [32]:
# Epsilon: Threshold value
epsilon = 0.5

In [35]:
for index, row in df1.iterrows():
    if row['avg_diff'] >= epsilon:
        df1.set_value(index,'Predicted_price_change',1)
    elif (-epsilon) <= row['avg_diff'] < epsilon:
        df1.set_value(index,'Predicted_price_change',0)
    elif row['avg_diff'] < (-epsilon):
        df1.set_value(index,'Predicted_price_change',-1)

/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/Users/sumeetkotaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


In [37]:
accuracy = 0
for index, row in df1.iterrows():
    if row['sentiment_vader'] == row['Predicted_price_change']:
        accuracy += 1
    else:
        continue

In [38]:
# Accuracy for naive analysis
accuracy/len(df1)

0.2505